In [ ]:
from gensim.models import Word2Vec
from gensim.matutils import argsort
from numpy import array,asarray
import numpy as np

In [ ]:
label_embed = model.syn0

In [ ]:
fmodel_embed = fmodel.syn0

In [ ]:
class WARPModel:
    def __init__(self,label_vec_f,feature_vec_f,binary=False):
        label2vec = Word2Vec.load_word2vec_format(label_vec_f,binary=binary)
        self.label_embed =  label2vec.syn0
        self.dictionary = label2vec.index2word
        
        self.feat_embed = Word2Vec.load_word2vec_format(feature_vec_f,binary=binary).syn0
    def scores(self,X):
        return np.dot(np.sum(self.feat_embed[X],axis=0),self.label_embed.T)
    def label_rank(self,X):
        return map(lambda x:self.dictionary[x],argsort(self.scores(X),reverse=True))
    def batch_predict(self,Xs):
        return [self.label_rank(X)[:2] for X in Xs]
        

In [ ]:
model = WARPModel('/Users/mayk/working/figer/baseline/PLE/Results/warp_B.bin','/Users/mayk/working/figer/baseline/PLE/Results/warp_A.bin')

In [ ]:
def read_data(fn):
    data = []
    with open(fn,'r') as f:
        for ln in f:
            data.append(map(int,ln.rstrip().split()[1].split(',')))
    return data

In [ ]:
data=read_data('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/test_x.txt')

In [ ]:
ls = model.label_rank(data[4])[:3]

In [ ]:
def refine(labels,maxDepth=2,delim='/'):
    keep = [""]*maxDepth
    for l in labels:
        path = getPath(l,delim)
        for i in range(len(path)):
            if keep[i] =="" and path[i]!="":
                keep[i] = path[i]
    results = []
    tmp= ''
    for l in keep:
        if l!="":
            tmp+=delim
            tmp +=l
            results.append(tmp)
        
    return results        
def getPath(label,delim='/'):
    return label.split(delim)[1:] 

In [10]:
from DataIO import *
import json

In [5]:
raw_data='../../Data/BBN/'
outdir='../../Results/'
data='../../Intermediate/BBN/'
output = outdir+'/predictionInText_warp.txt'
type_file = data + '/type.txt'
mention_file = data + '/mention.txt'
json_file = raw_data + '/test.json'
test_y_file = outdir+ 'warp_predictions'

In [6]:
mention_mapping = load_map(mention_file, 'mention')
label_mapping = load_map(type_file, 'label')
clean_mentions = load_mention_type(test_y_file)


In [8]:
def casestudy(filename, output, mention_mapping, label_mapping, clean_mentions):
    with open(filename) as f, open(output, 'w') as g:
        for line in f:
            sent = json.loads(line.strip('\r\n'))
            result = putback(sent, mention_mapping, label_mapping, clean_mentions)
            if result is not '':
                g.write(result+'\n')

In [15]:
casestudy(json_file, output, mention_mapping, label_mapping, clean_mentions)

In [12]:
def putback(sent_json, mention_mapping, label_mapping, clean_mentions):
    fileid = sent_json['fileid']
    senid = sent_json['senid']
    tokens = sent_json['tokens']
    pivot = 0
    result = []
    mentions = sent_json['mentions']
    sorted_m = sorted(mentions, cmp=compare)
    for m in sorted_m:
        start = m['start']
        end = m['end']
        if end - start == 1:
            mention_name = '[%s]' % (tokens[start])
        else:
            mention_name = '[%s]' % (' '.join(tokens[start:end]))
        if pivot <= start:
            result.extend(tokens[pivot:start])
            result.append(mention_name)
            # find predicted labels if any
            m_name = '%s_%d_%d_%d'%(fileid, senid, start, end)
            if m_name in mention_mapping:
                m_id = mention_mapping[m_name]
                if m_id in clean_mentions:
                    clean_labels = [label_mapping[l] for l in clean_mentions[m_id]]
                    result.append(':'+'['+','.join(clean_labels)+']')
        pivot = end
    if pivot < len(tokens):
        result.extend(tokens[pivot:])
    result = ' '.join([x for x in result if x is not None])
    return fileid+':'+str(senid)+'\t'+result + '\n'

In [14]:
def compare(item1, item2):
    if item1['start'] != item2['start']:
        return item1['start'] - item2['start']
    else:
        return item2['end'] - item1['end']

In [16]:
from collections import defaultdict
data = '/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/'
id2mention = defaultdict()
mention2id= defaultdict()
with open(data+'mention.txt','rb') as mf:
    for ln in mf:
        mid,iid = ln.rstrip().split()
        mention2id[mid]=iid   
        id2mention[iid]=mid

In [17]:
def convert(test_y_fn,idmap,out_fn):
    out = open(out_fn,'wb')
    with open(test_y_fn,'r') as Ys:
        for ln in Ys:
            mid,labels = ln.rstrip().split('\t')
            mid = idmap[mid]
            for l in labels.split(','):
                out.write('%s\t%s\t1\n' % (mid,l))
    out.close()

In [18]:
convert('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/test_y.txt',mention2id,'/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/test_y_new.txt')